# Démonstration : Attaques DoS/DDoS et Défenses

Ce notebook simule des attaques par déni de service (DoS/DDoS) et présente les mécanismes de défense.

⚠️ **AVERTISSEMENT** : Ce contenu est à but éducatif uniquement. Les attaques DoS/DDoS sont **illégales** sans autorisation explicite.

## 📚 Concepts couverts

1. **SYN Flood Attack** (simulation)
2. **Amplification Attacks** (DNS, NTP)
3. **Défenses** : SYN Cookies, Rate Limiting, Filtering
4. **Détection** d'anomalies de trafic

---

In [ ]:
import time
import random
from collections import deque, defaultdict
from typing import List, Tuple
import matplotlib.pyplot as plt
import numpy as np

## 1. Simulation SYN Flood Attack

### Principe

Une attaque **SYN Flood** exploite le handshake TCP en trois étapes :

```
1. Client → Server : SYN
2. Server → Client : SYN-ACK (serveur alloue ressources)
3. Client → Server : ACK (connexion établie)
```

**Attaque** : L'attaquant envoie des milliers de paquets SYN avec des adresses IP sources forgées, mais ne répond jamais avec l'ACK final. Le serveur garde des connexions semi-ouvertes jusqu'à timeout, épuisant ses ressources.

### Simulation

In [ ]:
class TCPServer:
    """Serveur TCP simulé vulnérable au SYN Flood"""
    
    def __init__(self, max_connections=100, timeout=5.0):
        self.max_connections = max_connections
        self.timeout = timeout
        # Connexions semi-ouvertes : {(src_ip, src_port): timestamp}
        self.half_open_connections = {}
        # Connexions établies
        self.established_connections = set()
        # Statistiques
        self.syn_received = 0
        self.connections_established = 0
        self.connections_refused = 0
    
    def receive_syn(self, src_ip: str, src_port: int) -> bool:
        """Reçoit un paquet SYN"""
        self.syn_received += 1
        
        # Nettoie les connexions expirées
        self._cleanup_expired()
        
        # Vérifie si on peut accepter la connexion
        if len(self.half_open_connections) >= self.max_connections:
            self.connections_refused += 1
            return False  # Table de connexions pleine
        
        # Ajoute la connexion semi-ouverte
        conn_id = (src_ip, src_port)
        self.half_open_connections[conn_id] = time.time()
        
        # Envoie SYN-ACK (simulé)
        return True
    
    def receive_ack(self, src_ip: str, src_port: int) -> bool:
        """Reçoit le ACK final pour établir la connexion"""
        conn_id = (src_ip, src_port)
        
        if conn_id in self.half_open_connections:
            # Connexion établie
            del self.half_open_connections[conn_id]
            self.established_connections.add(conn_id)
            self.connections_established += 1
            return True
        
        return False
    
    def _cleanup_expired(self):
        """Supprime les connexions expirées"""
        current_time = time.time()
        expired = [
            conn_id for conn_id, timestamp in self.half_open_connections.items()
            if current_time - timestamp > self.timeout
        ]
        for conn_id in expired:
            del self.half_open_connections[conn_id]
    
    def get_stats(self) -> dict:
        """Retourne les statistiques du serveur"""
        return {
            'syn_received': self.syn_received,
            'half_open': len(self.half_open_connections),
            'established': len(self.established_connections),
            'connections_established': self.connections_established,
            'connections_refused': self.connections_refused
        }

In [ ]:
def simulate_syn_flood(server: TCPServer, num_packets: int, legitimate_ratio: float = 0.1):
    """Simule une attaque SYN Flood"""
    print(f"🎯 Simulation SYN Flood : {num_packets} paquets")
    print(f"   Ratio légitime : {legitimate_ratio*100:.0f}%\n")
    
    for i in range(num_packets):
        # Génère une adresse IP aléatoire (attaque avec spoofing)
        src_ip = f"{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}"
        src_port = random.randint(10000, 65535)
        
        # Envoie le SYN
        accepted = server.receive_syn(src_ip, src_port)
        
        # Les connexions légitimes envoient l'ACK final
        if accepted and random.random() < legitimate_ratio:
            server.receive_ack(src_ip, src_port)
        
        # Simule un délai entre les paquets
        if i % 100 == 0:
            time.sleep(0.01)  # 10ms
    
    stats = server.get_stats()
    print("\n📊 Résultats :")
    print(f"   SYN reçus : {stats['syn_received']}")
    print(f"   Connexions semi-ouvertes : {stats['half_open']}")
    print(f"   Connexions établies : {stats['connections_established']}")
    print(f"   Connexions refusées : {stats['connections_refused']}")
    
    if stats['connections_refused'] > 0:
        print("\n❌ ATTAQUE RÉUSSIE : Le serveur a refusé des connexions (DoS)")
    else:
        print("\n✅ Le serveur a résisté à l'attaque")

In [ ]:
# Test 1 : Serveur vulnérable avec faible capacité
print("=" * 60)
print("TEST 1: Serveur vulnérable (max 100 connexions)")
print("=" * 60)

vulnerable_server = TCPServer(max_connections=100, timeout=5.0)
simulate_syn_flood(vulnerable_server, num_packets=500, legitimate_ratio=0.05)

# Test 2 : Serveur avec capacité augmentée
print("\n" + "=" * 60)
print("TEST 2: Serveur avec plus de capacité (max 1000 connexions)")
print("=" * 60)

robust_server = TCPServer(max_connections=1000, timeout=5.0)
simulate_syn_flood(robust_server, num_packets=500, legitimate_ratio=0.05)

## 2. Défense : SYN Cookies

### Principe

Les **SYN Cookies** évitent de stocker l'état des connexions semi-ouvertes. Le serveur encode l'information dans le numéro de séquence du SYN-ACK :

```
seq_number = Hash(src_ip, src_port, dst_ip, dst_port, timestamp, secret)
```

Lorsque le client renvoie l'ACK, le serveur peut vérifier et reconstruire l'état sans avoir stocké la connexion.

In [ ]:
import hashlib

class TCPServerWithSynCookies:
    """Serveur TCP avec protection SYN Cookies"""
    
    def __init__(self):
        self.secret = random.randbytes(32)
        self.established_connections = set()
        # Statistiques
        self.syn_received = 0
        self.connections_established = 0
        self.invalid_acks = 0
    
    def _generate_cookie(self, src_ip: str, src_port: int, timestamp: int) -> int:
        """Génère un SYN cookie"""
        data = f"{src_ip}:{src_port}:{timestamp}".encode()
        h = hashlib.sha256(self.secret + data).digest()
        # Prend les 4 premiers bytes comme cookie
        cookie = int.from_bytes(h[:4], 'big')
        return cookie
    
    def receive_syn(self, src_ip: str, src_port: int) -> Tuple[bool, int]:
        """Reçoit un SYN et génère un cookie"""
        self.syn_received += 1
        
        # Génère un cookie basé sur le timestamp actuel
        timestamp = int(time.time() / 60)  # Window de 60 secondes
        cookie = self._generate_cookie(src_ip, src_port, timestamp)
        
        # Retourne le cookie (qui sera le seq_number du SYN-ACK)
        return True, cookie
    
    def receive_ack(self, src_ip: str, src_port: int, ack_number: int) -> bool:
        """Vérifie l'ACK avec le cookie"""
        # Vérifie le cookie pour la fenêtre actuelle et précédente
        current_time = int(time.time() / 60)
        
        for timestamp in [current_time, current_time - 1]:
            expected_cookie = self._generate_cookie(src_ip, src_port, timestamp)
            
            if ack_number == expected_cookie + 1:  # ACK = SEQ + 1
                conn_id = (src_ip, src_port)
                self.established_connections.add(conn_id)
                self.connections_established += 1
                return True
        
        self.invalid_acks += 1
        return False
    
    def get_stats(self) -> dict:
        return {
            'syn_received': self.syn_received,
            'established': len(self.established_connections),
            'connections_established': self.connections_established,
            'invalid_acks': self.invalid_acks
        }

In [ ]:
def simulate_syn_flood_with_cookies(server: TCPServerWithSynCookies, num_packets: int, legitimate_ratio: float = 0.1):
    """Simule une attaque SYN Flood contre un serveur avec SYN Cookies"""
    print(f"🎯 Simulation SYN Flood avec SYN Cookies : {num_packets} paquets")
    print(f"   Ratio légitime : {legitimate_ratio*100:.0f}%\n")
    
    for i in range(num_packets):
        src_ip = f"{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}.{random.randint(1,255)}"
        src_port = random.randint(10000, 65535)
        
        # Envoie le SYN et reçoit le cookie
        accepted, cookie = server.receive_syn(src_ip, src_port)
        
        # Les connexions légitimes envoient l'ACK avec le bon cookie
        if accepted and random.random() < legitimate_ratio:
            ack_number = cookie + 1
            server.receive_ack(src_ip, src_port, ack_number)
    
    stats = server.get_stats()
    print("\n📊 Résultats :")
    print(f"   SYN reçus : {stats['syn_received']}")
    print(f"   Connexions établies : {stats['connections_established']}")
    print(f"   ACK invalides : {stats['invalid_acks']}")
    print("\n✅ Serveur protégé : Aucune ressource épuisée (pas de stockage d'état)")

# Test avec SYN Cookies
print("=" * 60)
print("TEST 3: Serveur avec SYN Cookies")
print("=" * 60)

protected_server = TCPServerWithSynCookies()
simulate_syn_flood_with_cookies(protected_server, num_packets=1000, legitimate_ratio=0.05)

## 3. Amplification Attack Simulation

### Principe

Une **attaque par amplification** exploite des serveurs tiers (DNS, NTP, etc.) pour amplifier le trafic vers la victime :

```
1. Attaquant envoie une petite requête (64 bytes) avec IP source forgée (victime)
2. Serveur DNS répond avec une grosse réponse (3000+ bytes) à la victime
3. Facteur d'amplification : 3000/64 ≈ 47x
```

### Calcul du facteur d'amplification

In [ ]:
class AmplificationAttack:
    """Calcule l'amplification de différents protocoles"""
    
    # Tailles typiques (en bytes)
    PROTOCOLS = {
        'DNS': {'request': 60, 'response': 3000, 'description': 'ANY query'},
        'NTP': {'request': 48, 'response': 468, 'description': 'monlist command'},
        'SSDP': {'request': 120, 'response': 1500, 'description': 'SSDP discovery'},
        'Memcached': {'request': 15, 'response': 750000, 'description': 'stats command'},
        'SNMP': {'request': 150, 'response': 1500, 'description': 'GetBulk request'},
    }
    
    @staticmethod
    def calculate_amplification(protocol: str) -> dict:
        """Calcule le facteur d'amplification"""
        data = AmplificationAttack.PROTOCOLS[protocol]
        factor = data['response'] / data['request']
        
        return {
            'protocol': protocol,
            'request_size': data['request'],
            'response_size': data['response'],
            'factor': factor,
            'description': data['description']
        }
    
    @staticmethod
    def simulate_attack(protocol: str, num_reflectors: int, attacker_bandwidth_mbps: float):
        """Simule une attaque par amplification"""
        amp = AmplificationAttack.calculate_amplification(protocol)
        
        # Bande passante de l'attaquant en bytes/s
        attacker_bw_bytes = (attacker_bandwidth_mbps * 1_000_000) / 8
        
        # Nombre de requêtes par seconde
        requests_per_sec = attacker_bw_bytes / amp['request_size']
        
        # Trafic total vers la victime (amplifié)
        victim_traffic_bytes = requests_per_sec * amp['response_size']
        victim_traffic_mbps = (victim_traffic_bytes * 8) / 1_000_000
        
        # Avec plusieurs reflectors
        victim_traffic_total_mbps = victim_traffic_mbps * num_reflectors
        
        return {
            'attacker_bw_mbps': attacker_bandwidth_mbps,
            'num_reflectors': num_reflectors,
            'requests_per_sec': requests_per_sec,
            'victim_traffic_mbps': victim_traffic_total_mbps,
            'amplification_factor': amp['factor']
        }

# Analyse des différents protocoles
print("=" * 70)
print("FACTEURS D'AMPLIFICATION PAR PROTOCOLE")
print("=" * 70)

for protocol in AmplificationAttack.PROTOCOLS.keys():
    amp = AmplificationAttack.calculate_amplification(protocol)
    print(f"\n{protocol:12s} | {amp['description']}")
    print(f"  Requête  : {amp['request_size']:8d} bytes")
    print(f"  Réponse  : {amp['response_size']:8d} bytes")
    print(f"  Facteur  : {amp['factor']:8.1f}x")

# Simulation d'attaque
print("\n" + "=" * 70)
print("SIMULATION: Attaque DNS Amplification")
print("=" * 70)

result = AmplificationAttack.simulate_attack(
    protocol='DNS',
    num_reflectors=1000,
    attacker_bandwidth_mbps=10
)

print(f"\nScénario :")
print(f"  Bande passante attaquant : {result['attacker_bw_mbps']:.0f} Mbps")
print(f"  Nombre de réflecteurs    : {result['num_reflectors']}")
print(f"  Facteur d'amplification  : {result['amplification_factor']:.1f}x")
print(f"\n🎯 Résultat :")
print(f"  Requêtes par seconde     : {result['requests_per_sec']:,.0f}")
print(f"  Trafic vers la victime   : {result['victim_traffic_mbps']:,.0f} Mbps ({result['victim_traffic_mbps']/1000:.1f} Gbps)")
print(f"\n❌ Impact : L'attaquant avec seulement 10 Mbps génère {result['victim_traffic_mbps']/result['attacker_bw_mbps']:.0f}x plus de trafic!")

## 4. Rate Limiting (Défense)

Le **rate limiting** limite le nombre de requêtes par IP/utilisateur dans une fenêtre de temps.

In [ ]:
class RateLimiter:
    """Implemente un rate limiter avec sliding window"""
    
    def __init__(self, max_requests: int, window_seconds: float):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        # {client_id: deque[timestamps]}
        self.requests = defaultdict(deque)
    
    def allow_request(self, client_id: str) -> bool:
        """Vérifie si la requête est autorisée"""
        current_time = time.time()
        client_requests = self.requests[client_id]
        
        # Supprime les requêtes hors de la fenêtre
        while client_requests and current_time - client_requests[0] > self.window_seconds:
            client_requests.popleft()
        
        # Vérifie si on peut accepter
        if len(client_requests) < self.max_requests:
            client_requests.append(current_time)
            return True
        
        return False

# Démonstration
print("=" * 60)
print("DÉMONSTRATION: Rate Limiting")
print("=" * 60)
print("Configuration: Max 10 requêtes par 5 secondes\n")

limiter = RateLimiter(max_requests=10, window_seconds=5.0)

# Client légitime
print("--- Client légitime (taux normal) ---")
for i in range(15):
    allowed = limiter.allow_request("client_1")
    status = "✅ Autorisée" if allowed else "❌ Bloquée"
    print(f"Requête {i+1:2d}: {status}")
    time.sleep(0.3)  # 300ms entre requêtes

# Attaquant (burst)
print("\n--- Attaquant (burst rapide) ---")
allowed_count = 0
blocked_count = 0

for i in range(100):
    allowed = limiter.allow_request("attacker")
    if allowed:
        allowed_count += 1
    else:
        blocked_count += 1

print(f"Autorisées : {allowed_count}")
print(f"Bloquées   : {blocked_count}")
print(f"\n✅ Rate limiting effectif : {blocked_count/100*100:.0f}% des requêtes bloquées")

## 5. Détection d'anomalies de trafic

Détection d'une attaque DoS par analyse statistique du trafic.

In [ ]:
def generate_traffic(duration_seconds: int, attack_start: int, attack_duration: int):
    """Génère un trafic simulé avec une période d'attaque"""
    traffic = []
    
    for second in range(duration_seconds):
        if attack_start <= second < attack_start + attack_duration:
            # Pendant l'attaque : trafic très élevé
            packets = random.randint(8000, 12000)
        else:
            # Trafic normal : avec variation
            baseline = 1000
            variation = random.randint(-200, 200)
            packets = baseline + variation
        
        traffic.append(packets)
    
    return traffic

def detect_anomalies(traffic: List[int], threshold_std: float = 3.0) -> List[int]:
    """Détecte les anomalies avec la méthode du Z-score"""
    mean = np.mean(traffic)
    std = np.std(traffic)
    
    anomalies = []
    for i, value in enumerate(traffic):
        z_score = (value - mean) / std
        if abs(z_score) > threshold_std:
            anomalies.append(i)
    
    return anomalies, mean, std

# Génère et analyse le trafic
print("=" * 60)
print("DÉTECTION D'ANOMALIES DE TRAFIC")
print("=" * 60)

duration = 120  # 2 minutes
attack_start = 60
attack_duration = 20

traffic = generate_traffic(duration, attack_start, attack_duration)
anomalies, mean, std = detect_anomalies(traffic, threshold_std=3.0)

print(f"\nStatistiques du trafic :")
print(f"  Moyenne : {mean:.0f} paquets/s")
print(f"  Écart-type : {std:.0f} paquets/s")
print(f"  Seuil d'alerte : {mean + 3*std:.0f} paquets/s")
print(f"\nAnomalies détectées : {len(anomalies)} secondes")

if anomalies:
    print(f"Période d'attaque détectée : secondes {min(anomalies)} à {max(anomalies)}")
    print(f"Période d'attaque réelle   : secondes {attack_start} à {attack_start + attack_duration}")
    
    # Calcule la précision de la détection
    true_attack_seconds = set(range(attack_start, attack_start + attack_duration))
    detected_seconds = set(anomalies)
    true_positives = len(true_attack_seconds & detected_seconds)
    precision = true_positives / len(detected_seconds) if detected_seconds else 0
    recall = true_positives / len(true_attack_seconds) if true_attack_seconds else 0
    
    print(f"\n📊 Précision de détection :")
    print(f"   Precision : {precision*100:.1f}%")
    print(f"   Recall    : {recall*100:.1f}%")

# Visualisation
plt.figure(figsize=(12, 6))
plt.plot(traffic, label='Trafic', linewidth=1)
plt.axhline(y=mean, color='g', linestyle='--', label='Moyenne', alpha=0.7)
plt.axhline(y=mean + 3*std, color='r', linestyle='--', label='Seuil (3σ)', alpha=0.7)
plt.axvline(x=attack_start, color='orange', linestyle=':', label='Début attaque', alpha=0.7)
plt.axvline(x=attack_start + attack_duration, color='orange', linestyle=':', label='Fin attaque', alpha=0.7)

# Marque les anomalies
if anomalies:
    plt.scatter(anomalies, [traffic[i] for i in anomalies], 
                color='red', s=50, label='Anomalies détectées', zorder=5)

plt.xlabel('Temps (secondes)')
plt.ylabel('Paquets par seconde')
plt.title('Détection d\'attaque DoS par analyse de trafic')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Anomalie détectée avec succès!")

## 📊 Résumé des défenses contre DoS/DDoS

### ✅ Défenses au niveau réseau

1. **SYN Cookies** : Évite l'épuisement de la table de connexions TCP
2. **Rate Limiting** : Limite les requêtes par IP/client
3. **IP Filtering** : Bloque les adresses IP suspectes
4. **Geo-blocking** : Bloque les pays sources d'attaques
5. **BPF (Berkeley Packet Filter)** : Filtrage au niveau kernel

### ✅ Défenses au niveau infrastructure

1. **CDN** (Cloudflare, Akamai) : Distribue le trafic géographiquement
2. **Scrubbing Centers** : Nettoie le trafic malveillant avant qu'il atteigne la cible
3. **Anycast** : Distribue les requêtes sur plusieurs serveurs
4. **Load Balancing** : Répartit la charge sur plusieurs machines

### ✅ Détection et mitigation

1. **IDS/IPS** : Détecte les patterns d'attaque
2. **NetFlow Analysis** : Analyse statistique du trafic
3. **Machine Learning** : Détection d'anomalies
4. **CAPTCHA** : Distingue humains et bots
5. **Blackholing** : Redirige le trafic malveillant vers un null route

---

## ⚠️ Limitations

Aucune défense n'est parfaite contre les DDoS massifs :
- **Attaques volumétriques** (Tbps) peuvent saturer même les CDN
- **Coût** : Mitigation professionnelle très coûteuse
- **Faux positifs** : Le rate limiting peut bloquer des utilisateurs légitimes
- **Attaques ciblées** : Difficile de distinguer légitime vs malveillant

---

## 🔗 Références

- **RFC 4987** : TCP SYN Flooding Attacks and Common Mitigations
- **Cloudflare DDoS Reports** : https://blog.cloudflare.com/tag/ddos/
- **OWASP** : Denial of Service Cheat Sheet